# HuggingFace Datasets and Tigris

In order to get started with HuggingFace Datasets Tigris in an iPython notebook, you need the following:

1. A Tigris account
1. An access keypair
1. A computer running Python 3.10 (or later) that has internet access (OS and CPU architecture does not matter)
1. A tigris bucket
1. The uv python environment manager

If you us [VS Code](https://code.visualstudio.com/) and have the [Development Containers](https://code.visualstudio.com/docs/devcontainers/containers) extension installed, clone this repository to your machine and run the command `Dev Containers: Reopen in Container`. This will automatically set up all of the dependencies you need to get started.

Install all of the dependencies for this project with `uv`:

In [4]:
! uv python install 3.10
! uv venv
! uv sync

Using CPython 3.10.18
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate
Resolved 75 packages in 1ms
⠙ Preparing packages... (0/33)                                                  
⠙ Preparing packages... (0/33)-------------     0 B/126.75 KiB          
⠙ Preparing packages... (0/33)-------------     0 B/126.75 KiB          
packaging            ------------------------------     0 B/64.91 KiB
⠙ Preparing packages... (0/33)-------------     0 B/126.75 KiB          
packaging            ------------------------------     0 B/64.91 KiB
⠙ Preparing packages... (0/33)-------------     0 B/126.75 KiB          
packaging            ------------------------------     0 B/64.91 KiB
⠙ Preparing packages... (0/33)-------------     0 B/126.75 KiB          
attrs                ------------------------------     0 B/62.32 KiB
packaging            ------------------------------     0 B/64.91 KiB
⠙ Preparing packages... (0/33)-------------     0 B/126.75 KiB          
a

Copy the contents of `.env.example` into `.env` and open `.env` in your editor.

If you are not using VS Code for this, you will need to open `.env` in your editor manually.

In [ ]:
! cp .env.example .env
! code .env || echo "open .env in your editor"

Put your access key in the `AWS_ACCESS_KEY_ID` field and put your secret key in the `AWS_SECRET_ACCESS_KEY` field.

For example, if your access key is `tid_NotKbzPHpJuoX` and your secret access key is `tsec_r++Q9iocfdf7Th`:

```patch
 ## Tigris configuration

 # Change these based on the access key you got from the web console
-AWS_ACCESS_KEY_ID=tid_access_key_id
-AWS_SECRET_ACCESS_KEY=tsec_secret_access_key
+AWS_ACCESS_KEY_ID=tid_NotKbzPHpJuoX
+AWS_SECRET_ACCESS_KEY=tsec_r++Q9iocfdf7Th
```

Then load the `.env` file into your notebook's environment:

In [8]:
from dotenv import load_dotenv

load_dotenv()

True

Then make sure you got everything:

In [9]:
import os

dotenv_errs = []
for key in [
    "AWS_ACCESS_KEY_ID",
    "AWS_SECRET_ACCESS_KEY",
    "AWS_ENDPOINT_URL_S3",
    "AWS_ENDPOINT_URL_IAM",
    "AWS_REGION",
]:
    assert os.getenv(key) is not None, f"Environment variable {key} is not defined, please define it in .env"

Then set up the storage options for the datasets library:

In [10]:
storage_options = {
    "key": os.getenv("AWS_ACCESS_KEY_ID"),
    "secret": os.getenv("AWS_SECRET_ACCESS_KEY"),
    "endpoint_url": os.getenv("AWS_ENDPOINT_URL_S3"),
}

Make sure you have permissions to write files to your bucket:

In [13]:
import s3fs


# Change me!
bucket_name = "xe-datasets-demo"


fs = s3fs.S3FileSystem(**storage_options)
fs.write_text(f"/{bucket_name}/test.txt", "this is a test")
fs.rm(f"/{bucket_name}/test.txt")

[]

Load a dataset such as [mlabonne/FineTome-100k](http://hf.co/datasets/mlabonne/FineTome-100k):

In [14]:
from datasets import load_dataset
from IPython.display import display


dataset_name = "mlabonne/FineTome-100k"


dataset = load_dataset(dataset_name, split="train")
display(dataset)

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

Then copy it to your bucket:

In [17]:
dataset.save_to_disk(
    f"s3://{bucket_name}/datasets/{dataset_name}",
    storage_options=storage_options,
)

Saving the dataset (0/1 shards): 100%|██████████| 100000/100000 [00:11<00:00, 8914.79 examples/s]


FileNotFoundError: The specified multipart upload does not exist. The upload ID may be invalid, or the upload may have been aborted or completed.

Then you can import it from Tigris in another workflow:

In [6]:
dataset = load_dataset(f"s3://{bucket_name}/datasets/{dataset_name}", storage_options=storage_options)


def remove_blue(row):
    display(row)
    assert False


filtered_ds = dataset.filter(remove_blue)

NameError: name 'bucket_name' is not defined